## crawling한 korean books에서 중복되는 책들을 삭제, 에디션만 다른 동일한 내용의 책 삭제

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.appName("spark").getOrCreate()

# create the Spark Context
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/14 14:46:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/14 14:46:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("SELECT title, author FROM Korean_book")
data = cur.fetchall()
print(len(data))
conn.commit()
conn.close()


44945


In [3]:
## 중복 도서 삭제 확인
rdd = sc.parallelize(data)
S = set({})
D = set({})
for d in rdd.collect():
    if d in S:
        D.add(d[0])
    else:
        S.add(d[0])

print(len(S))
print(len(D)) # <--- 실제 중복 데이터(책 제목과 글쓴이도 다른)
real_dup_data = list(D)

44616
0


In [ ]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()

# 하나 빼고 중복되는 도서들 모두 삭제
for i in range(len(real_dup_data)):
    cur.execute(f"SELECT rowid FROM Korean_book WHERE title=\"{real_dup_data[i]}\"")
    same = cur.fetchall()
    del_id = []
    for num, (item) in enumerate(same):
        if num==0:
            pass
        else:
            del_id.append(item[0])

    for id in del_id:
        cur.execute(f"DELETE FROM Korean_book WHERE rowid={id}")
    conn.commit()
    cur.execute(f"SELECT rowid FROM Korean_book WHERE title=\"{real_dup_data[i]}\"")
    same = cur.fetchall()

conn.close()

In [3]:
## 에디션만 다른 같은 이름의 도서 check
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("SELECT rowid, title, author FROM Korean_book")
data = cur.fetchall()
# print(data[0])

for id, title, author in data:
    for i, t, a in data:
        if id!=i and author==a:
            if (title in t) or (t in title):
                print(f"{id:-6}\t{title}\t{author}")
                print(f"{i:-6}\t{t}\t{a}")
                print("-"*100)
            else:
                pass


    10	아몬드(100만 부 기념 특별판)	손원평	★★★ 아시아권 최초, 2020 일본 서점대상 번역소설 부문 수상
★★★ 미국, 프랑스, 일본, 중국, 스페인, 태국 등 20여 개국 번역 수출 계약
★★★ 2017 교보문고 올해의 한국소설, 2021 영풍문고 올해의 책 
★★★ 2022 뮤지컬, 연극으로 재탄생 

아시아를 넘은 세계문학의 명작, 모든 세대에게 사랑받을 작품. 일본 서점대상
청소년과 성인 독자들에게 널리 알려져야 할 경이로운 책. 월스트리트저널 리뷰

2017년 출간되어 꾸준히 사랑받아 온 손원평 작가의 첫 장편소설 『아몬드』가 출간 5년 만에 국내 판매 100만 부를 맞아 특별판을 출간한다. 전 세계, 전 세대 독자들에게 꾸준히 큰 감동을 전하며 한국 영어덜트 소설의 기준을 세운 『아몬드』 ‘100만 부 기념 특별판’에는 ‘다시 쓰는 작가의 말’이 수록되었으며, 새로운 일러스트가 담긴 표지로 단장하여 독자들의 소장 욕구를 자극한다. 
『아몬드』는 감정을 느끼지 못하는 소년의 특별한 성장 이야기로, 첫 장을 읽기 시작하면 끝까지 놓을 수 없는 흡인력 강한 작품이다. 인물들이 타인과 관계 맺고 슬픔에 공감하며 성장해 나가는 과정을 탁월하게 묘사했다는 평을 얻었으며, 영화처럼 펼쳐지는 극적인 사건과 매혹적인 문체가 돋보인다.
또한 『아몬드』는 지금까지 미국, 스페인, 일본 등 20여 개국으로 번역 수출되어 전 세계의 독자들을 만나고 있으며, 국내 및 해외에서 뮤지컬과 연극으로도 재탄생된 바 있다. 지난 2020년에는 아시아권 최초로 일본 서점대상의 번역소설 부문을 수상하면서 “아시아를 넘은 세계문학의 명작”으로 자리매김했다. ‘감정’이라는 보편적인 주제로 동시대를 살아가는 모든 사람의 마음을 사로잡는 책이다.
    30	아몬드	손원평	영화와도 같은 강렬한 사건과 매혹적인 문체로 시선을 사로잡는 한국형 영 어덜트 소설 『아몬드』. 타인의 감정에 무감각해진 공감 불능인 이 시대에 큰 울림을 주는 이 작품은 감정을 느끼지 못하는 한 소년의 특별한